In [ ]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path=r"D:\UET courses\IOT and Application\Fall_detection\model_final\mobilenetv2_qat_int8.tflite")
interpreter.allocate_tensors()

import numpy as np
from PIL import Image
import time
import os
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import time

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load ảnh
img = Image.open("D:/UET courses/IOT and Application/Fall_detection/Data/check_image/image001.jpg").convert("RGB")

# Resize theo input của mô hình
input_shape = input_details[0]['shape']  # [1, H, W, C]
img_resized = img.resize((input_shape[2], input_shape[1]))  # width, height

# Chuyển sang numpy array
input_data = np.array(img_resized)

# Kiểm tra dtype của input
input_dtype = input_details[0]['dtype']
if input_dtype == np.int8 or input_dtype == np.uint8:
    scale, zero_point = input_details[0]['quantization']
    input_data = input_data / 255.0           # normalize 0..1
    input_data = input_data / scale + zero_point
    input_data = input_data.astype(input_dtype)
else:
    input_data = input_data.astype(np.float32) / 255.0

# Thêm batch dimension
input_data = np.expand_dims(input_data, axis=0)

# Inference và đo thời gian
start_time = time.time()
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
end_time = time.time()

# Xử lý output
if output.shape[-1] == 1:  # sigmoid binary
    pred_class = 1 if output[0][0] > 0.5 else 0
else:                      # softmax
    pred_class = np.argmax(output)

classes = ["No Fall", "Fall"]

print(f"Output raw: {output}")
print(f"Predicted class: {classes[pred_class]}")
print(f"Inference time: {(end_time-start_time)*1000:.2f} ms")


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Output raw: [[99]]
Predicted class: Fall
Inference time: 5.00 ms


In [ ]:


# Cấu hình
IMG_SIZE = 128
DATA_DIR = r"D:\UET courses\IOT and Application\Fall_detection\data\images"
TFLITE_MODEL_PATH = r"D:\UET courses\IOT and Application\Fall_detection\model_final\mobilenetv2_qat_int8.tflite"

# =============================================================================
# LOAD VALIDATION DATA
# =============================================================================

def load_validation_data(data_dir):
    """Load tất cả ảnh validation"""
    valid_dir = Path(data_dir) / 'valid'
    
    if not valid_dir.exists():
        raise ValueError(f"Thư mục valid không tồn tại: {valid_dir}")
    
    image_paths = []
    labels = []
    
    # Load tất cả ảnh
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
        image_paths.extend(valid_dir.glob(ext))
    
    # Xác định label từ tên file
    for img_path in image_paths:
        filename = img_path.stem.lower()
        if 'fall' in filename and 'not' not in filename:
            labels.append(1)  # Fall
        else:
            labels.append(0)  # Not fall
    
    print(f"\n{'='*60}")
    print(f"📁 VALIDATION SET INFO")
    print(f"{'='*60}")
    print(f"Total images: {len(image_paths)}")
    print(f"  - Fall: {sum(labels)} ({sum(labels)/len(labels)*100:.1f}%)")
    print(f"  - Not Fall: {len(labels)-sum(labels)} ({(len(labels)-sum(labels))/len(labels)*100:.1f}%)")
    print(f"{'='*60}\n")
    
    return [str(p) for p in image_paths], np.array(labels)

# =============================================================================
# LOAD TFLITE MODEL
# =============================================================================

def load_tflite_model(model_path):
    """Load TFLite model và lấy thông tin"""
    print(f"📦 Loading TFLite model: {model_path}")
    
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    
    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    print(f"\n🔍 MODEL INFO:")
    print(f"  Input shape: {input_details[0]['shape']}")
    print(f"  Input dtype: {input_details[0]['dtype']}")
    print(f"  Output shape: {output_details[0]['shape']}")
    print(f"  Output dtype: {output_details[0]['dtype']}")
    
    # Quantization info
    input_scale, input_zero_point = input_details[0]['quantization']
    output_scale, output_zero_point = output_details[0]['quantization']
    
    print(f"\n📊 QUANTIZATION INFO:")
    print(f"  Input scale: {input_scale}, zero_point: {input_zero_point}")
    print(f"  Output scale: {output_scale}, zero_point: {output_zero_point}")
    
    return interpreter, input_details, output_details

# =============================================================================
# INFERENCE FUNCTIONS
# =============================================================================

def preprocess_image(img_path, target_size=IMG_SIZE):
    """Tiền xử lý ảnh"""
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [target_size, target_size])
    img = img / 255.0
    return img.numpy()

def quantize_input(img, input_details):
    """Quantize input image"""
    input_scale, input_zero_point = input_details[0]['quantization']
    
    if input_details[0]['dtype'] == np.uint8:
        img_quantized = (img / input_scale + input_zero_point).astype(np.uint8)
    else:
        img_quantized = (img / input_scale + input_zero_point).astype(np.int8)
    
    return img_quantized

def dequantize_output(output, output_details):
    """Dequantize output"""
    output_scale, output_zero_point = output_details[0]['quantization']
    output_float = (output.astype(np.float32) - output_zero_point) * output_scale
    return output_float

def run_inference(interpreter, input_details, output_details, img_path):
    """Chạy inference cho 1 ảnh"""
    # Preprocess
    img = preprocess_image(img_path)
    
    # Quantize input
    img_quantized = quantize_input(img, input_details)
    
    # Run inference
    interpreter.set_tensor(input_details[0]['index'], [img_quantized])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    
    # Dequantize output
    output_float = dequantize_output(output, output_details)
    
    return output_float[0][0]

# =============================================================================
# EVALUATION
# =============================================================================

def evaluate_on_validation_set(interpreter, input_details, output_details, val_paths, val_labels):
    """Đánh giá trên toàn bộ validation set"""
    print(f"\n{'='*60}")
    print(f"🧪 EVALUATING ON VALIDATION SET")
    print(f"{'='*60}\n")
    
    predictions = []
    probabilities = []
    total = len(val_paths)
    
    print(f"Processing {total} images...")
    start_time = time.time()
    
    for i, img_path in enumerate(val_paths):
        if (i + 1) % 50 == 0:
            print(f"  Processed: {i+1}/{total} ({(i+1)/total*100:.1f}%)")
        
        prob = run_inference(interpreter, input_details, output_details, img_path)
        probabilities.append(prob)
        predictions.append(1 if prob > 0.5 else 0)
    
    end_time = time.time()
    inference_time = (end_time - start_time) / total * 1000  # ms per image
    
    predictions = np.array(predictions)
    probabilities = np.array(probabilities)
    
    # Calculate metrics
    accuracy = np.mean(predictions == val_labels)
    
    # Confusion matrix
    cm = confusion_matrix(val_labels, predictions)
    
    # Per-class metrics
    tn, fp, fn, tp = cm.ravel()
    
    precision_fall = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall_fall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_fall = 2 * (precision_fall * recall_fall) / (precision_fall + recall_fall) if (precision_fall + recall_fall) > 0 else 0
    
    precision_not_fall = tn / (tn + fn) if (tn + fn) > 0 else 0
    recall_not_fall = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1_not_fall = 2 * (precision_not_fall * recall_not_fall) / (precision_not_fall + recall_not_fall) if (precision_not_fall + recall_not_fall) > 0 else 0
    
    print(f"\n{'='*60}")
    print(f"📊 RESULTS")
    print(f"{'='*60}")
    print(f"\n⏱️  PERFORMANCE:")
    print(f"  Average inference time: {inference_time:.2f} ms/image")
    print(f"  Total time: {end_time - start_time:.2f} seconds")
    
    print(f"\n🎯 OVERALL METRICS:")
    print(f"  Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Correct predictions: {np.sum(predictions == val_labels)}/{total}")
    
    print(f"\n📈 PER-CLASS METRICS:")
    print(f"\n  NOT FALL (Class 0):")
    print(f"    Precision: {precision_not_fall:.4f}")
    print(f"    Recall: {recall_not_fall:.4f}")
    print(f"    F1-Score: {f1_not_fall:.4f}")
    
    print(f"\n  FALL (Class 1):")
    print(f"    Precision: {precision_fall:.4f}")
    print(f"    Recall: {recall_fall:.4f}")
    print(f"    F1-Score: {f1_fall:.4f}")
    
    print(f"\n🔢 CONFUSION MATRIX:")
    print(f"                Predicted")
    print(f"                Not Fall    Fall")
    print(f"  Actual Not Fall   {tn:4d}      {fp:4d}")
    print(f"  Actual Fall       {fn:4d}      {tp:4d}")
    
    print(f"\n{'='*60}\n")
    
    # Classification report
    print("📋 DETAILED CLASSIFICATION REPORT:")
    print(classification_report(val_labels, predictions, 
                                target_names=['Not Fall', 'Fall'],
                                digits=4))
    
    return {
        'accuracy': accuracy,
        'predictions': predictions,
        'probabilities': probabilities,
        'confusion_matrix': cm,
        'inference_time_ms': inference_time,
        'metrics': {
            'not_fall': {'precision': precision_not_fall, 'recall': recall_not_fall, 'f1': f1_not_fall},
            'fall': {'precision': precision_fall, 'recall': recall_fall, 'f1': f1_fall}
        }
    }

# =============================================================================
# VISUALIZATION
# =============================================================================

def plot_confusion_matrix(cm, filename='confusion_matrix.png'):
    """Vẽ confusion matrix"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Not Fall', 'Fall'],
                yticklabels=['Not Fall', 'Fall'],
                cbar_kws={'label': 'Count'})
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    print(f"  📊 Saved confusion matrix: {filename}")
    plt.close()


# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":
    # Load validation data
    val_paths, val_labels = load_validation_data(DATA_DIR)
    
    # Load TFLite model
    interpreter, input_details, output_details = load_tflite_model(TFLITE_MODEL_PATH)
    
    # Evaluate
    results = evaluate_on_validation_set(
        interpreter, input_details, output_details, val_paths, val_labels
    )
    
    # Visualize
    print(f"\n{'='*60}")
    print(f"📊 CREATING VISUALIZATIONS")
    print(f"{'='*60}\n")
    
    plot_confusion_matrix(results['confusion_matrix'])
    # plot_probability_distribution(results['probabilities'], val_labels)
    
    # Error analysis
    # analyze_errors(val_paths, val_labels, results['predictions'], results['probabilities'])
    
    # Summary
    print(f"\n{'='*60}")
    print(f"✅ EVALUATION COMPLETE")
    print(f"{'='*60}")
    print(f"\n🎯 Key Metrics:")
    print(f"  • Overall Accuracy: {results['accuracy']*100:.2f}%")
    print(f"  • Fall Detection F1: {results['metrics']['fall']['f1']:.4f}")
    print(f"  • Not Fall F1: {results['metrics']['not_fall']['f1']:.4f}")
    print(f"  • Inference Speed: {results['inference_time_ms']:.2f} ms/image")
    print(f"\n💾 Saved files:")
    print(f"  • confusion_matrix.png")
    print(f"  • probability_distribution.png")
    print(f"{'='*60}\n")


📁 VALIDATION SET INFO
Total images: 222
  - Fall: 140 (63.1%)
  - Not Fall: 82 (36.9%)

📦 Loading TFLite model: D:\UET courses\IOT and Application\Fall_detection\model_final\mobilenetv2_qat_int8.tflite

🔍 MODEL INFO:
  Input shape: [  1 128 128   3]
  Input dtype: <class 'numpy.uint8'>
  Output shape: [1 1]
  Output dtype: <class 'numpy.uint8'>

📊 QUANTIZATION INFO:
  Input scale: 0.003921568859368563, zero_point: 0
  Output scale: 0.00390625, zero_point: 0

🧪 EVALUATING ON VALIDATION SET

Processing 222 images...


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


  Processed: 50/222 (22.5%)
  Processed: 100/222 (45.0%)
  Processed: 150/222 (67.6%)
  Processed: 200/222 (90.1%)

📊 RESULTS

⏱️  PERFORMANCE:
  Average inference time: 15.43 ms/image
  Total time: 3.43 seconds

🎯 OVERALL METRICS:
  Accuracy: 0.7748 (77.48%)
  Correct predictions: 172/222

📈 PER-CLASS METRICS:

  NOT FALL (Class 0):
    Precision: 0.6333
    Recall: 0.9268
    F1-Score: 0.7525

  FALL (Class 1):
    Precision: 0.9412
    Recall: 0.6857
    F1-Score: 0.7934

🔢 CONFUSION MATRIX:
                Predicted
                Not Fall    Fall
  Actual Not Fall     76         6
  Actual Fall         44        96


📋 DETAILED CLASSIFICATION REPORT:
              precision    recall  f1-score   support

    Not Fall     0.6333    0.9268    0.7525        82
        Fall     0.9412    0.6857    0.7934       140

    accuracy                         0.7748       222
   macro avg     0.7873    0.8063    0.7729       222
weighted avg     0.8275    0.7748    0.7783       222


📊 CREAT

C:\Users\Admin\AppData\Local\Temp\ipykernel_14192\3159848076.py:262: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax2.boxplot(data_to_plot, labels=['Not Fall', 'Fall'])


  📊 Saved probability distribution: probability_distribution.png

🔍 ERROR ANALYSIS

Total errors: 50 / 222 (22.52%)

Error breakdown:
  False Positives (predicted Fall, actual Not Fall): 6
  False Negatives (predicted Not Fall, actual Fall): 44

📋 Top 10 errors (sorted by confidence):
Type            Confidence   File
--------------------------------------------------------------------------------
False Positive  0.8828       not fallen035.png
False Positive  0.8828       not fallen035.png
False Negative  0.8633       fall015.jpg
False Negative  0.8633       fall002.jpg
False Negative  0.8633       fall023.jpg
False Negative  0.8633       fall022.jpg
False Negative  0.8633       fall020.jpg
False Negative  0.8633       fall003.jpg
False Negative  0.8633       fall038.jpg
False Negative  0.8633       fall037.jpg

✅ EVALUATION COMPLETE

🎯 Key Metrics:
  • Overall Accuracy: 77.48%
  • Fall Detection F1: 0.7934
  • Not Fall F1: 0.7525
  • Inference Speed: 15.43 ms/image

💾 Saved files:
  •